In [6]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim.models import word2vec

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [7]:
# Read recipe inputs
model_folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = model_folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [8]:
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
reviews_TF_IDF_df = reviews_TF_IDF.get_dataframe()

In [9]:
tripAdvisor_path = dataiku.Folder("FXMfF0DU").get_path() + "/TripAdvisor_top_TFIDF_words.txt"
f = open(tripAdvisor_path,'r',encoding="utf-8")

In [11]:

for i,data in enumerate(f):
    print(data)

レストラン

私たち

あなた

彼ら

パリ

待ち行列

クール

快適

窮屈

事項

キリン

共有

パイ

宣伝

提案

バー

インテリア

コンパクト

ストック

親切



In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

recommendation_df = reviews_TF_IDF_df # For this sample code, simply copy input to output


# Write recipe outputs
recommendation = dataiku.Dataset("recommendation")
recommendation.write_with_schema(recommendation_df)